In [1]:
# =============================================
# 1. CONFIGURACIÓN INICIAL
# =============================================

import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Ruta al DW
db_path = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/data/external/chivas_dw.sqlite"

# Conexión
conn = sqlite3.connect(db_path)

# Cargar la vista combinada
df = pd.read_sql_query("SELECT * FROM vw_predicciones_diarias_extendida", conn)
# Cerrar conexión
conn.close()

print(f"Registros cargados: {len(df)}")
df.head()


Registros cargados: 2163


,id_jugador,Fecha,microciclo_next,tipo_dia_next,Tipo_Dia,Edad,Peso_kg,Estatura_cm,Posicion,Linea,...,Carga_Regenerativa,Distancia_total,HMLD_m,HSR_abs_m,Sprints_cantidad,Acc_3,Dec_3,Player_Load,RPE,Sprints_vel_max_kmh
0,25,2025-05-23,1,NaN,ENTRENO,26,70.0,176.0,Delantero,Extremo,...,1562.0,3294,170,0,0,7,1,49,4,18.3
1,25,2025-05-24,1,NaN,ENTRENO,26,70.0,176.0,Delantero,Extremo,...,1528.5,3707,512,138,0,8,6,60,5,22.1
2,25,2025-05-26,2,NaN,ENTRENO,26,70.0,176.0,Delantero,Extremo,...,998.5,2281,284,0,0,39,22,31,0,20.5
3,25,2025-05-27,2,NaN,ENTRENO,26,70.0,176.0,Delantero,Extremo,...,2020.5,4821,638,142,1,36,27,49,0,25.5
4,25,2025-05-28,2,NaN,ENTRENO,26,70.0,176.0,Delantero,Extremo,...,1808.5,4523,741,165,3,54,30,70,0,26.7


In [2]:
# Primero, como hicimos en todos los dataset a entrenar, excluimos todos los jugadores 
# que no proporcionan valor, al igual que los microciclos donde no se compitió:

# =============================================
# 2.1 FILTRO DE JUGADORES Y MICRO-CICLOS
# =============================================

print(f"Registros cargados pre filtrado: {len(df)}\n")

# Jugadores que deben excluirse
jugadores_excluir = [1, 2, 3, 12, 30]

# Filtrar jugadores no válidos
df = df[~df["id_jugador"].isin(jugadores_excluir)]

# fFiltramos los días nulos o no validos (elimina microciclos fuera de competencia):
df = df[df['tipo_dia_next'].notnull()]


print(f"Registros cargados post filtrado: {len(df)}\n")
# Distribución de la etiqueta
print("\nDistribución de tipo_semana_next:")
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)


Registros cargados pre filtrado: 2163

Registros cargados post filtrado: 1357


Distribución de tipo_semana_next:
tipo_semana_next
media    50.257922
alta     33.971997
baja     15.770081
Name: proportion, dtype: float64


In [3]:
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)
print(df['tipo_dia_next'].value_counts())


tipo_semana_next
media    50.257922
alta     33.971997
baja     15.770081
Name: proportion, dtype: float64
tipo_dia_next
-1.0    418
 1.0    301
-2.0    255
 2.0    249
 3.0    134
Name: count, dtype: int64


In [4]:
features = [
    'tipo_dia_next', 'tipo_semana_next',
    'CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual',
    'riesgo_suavizado_3d_actual',
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next'
]
target = 'Carga_Sostenida'


In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

le = LabelEncoder()
df['tipo_semana_next'] = le.fit_transform(df['tipo_semana_next'])

X = df[features]
y = df[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


R²: 0.43078548723009125
MAE: 194.7538644859837


In [7]:
from scipy.stats import spearmanr
corr, _ = spearmanr(y_test, y_pred)
print(f"📈 Correlación de rangos (Spearman): {corr:.3f}")


📈 Correlación de rangos (Spearman): 0.669


### El modelo predijo, con algunos detlles que se pueden corregir. En esta etapa se evaluó sin tener en cuenta al jugador ni su pocisión: Vamos agergar esos datos junto con un perfil fisiológico para darle más contexto al modelo de aprender con mayor especificidad .

In [8]:
# ============================================
# 🧠 CONTEXTO FISIOLÓGICO: jugador + posición
# ============================================
import pandas as pd

# 1️⃣ Reemplazar nulos en posición y línea por "Desconocido" (si los hay)
df['Posicion'] = df['Posicion'].fillna('Desconocido')
df['Linea'] = df['Linea'].fillna('Desconocido')

# 2️⃣ Calcular medias históricas por jugador
player_mean = df.groupby('id_jugador').agg({
    'Distancia_total': 'mean',
    'Player_Load': 'mean',
    'Acc_3': 'mean',
    'Dec_3': 'mean'
}).rename(columns={
    'Distancia_total': 'jugador_mean_dist',
    'Player_Load': 'jugador_mean_load',
    'Acc_3': 'jugador_mean_acc',
    'Dec_3': 'jugador_mean_dec'
}).reset_index()

# Unir las medias al dataframe principal
df = df.merge(player_mean, on='id_jugador', how='left')

# 3️⃣ Codificar posición y línea (one-hot encoding)
df = pd.get_dummies(df, columns=['Posicion', 'Linea'], prefix=['Pos', 'Lin'])

# 4️⃣ Confirmar que se agregaron bien las nuevas columnas
print("✅ Nuevas columnas añadidas:", [c for c in df.columns if 'Pos_' in c or 'Lin_' in c])
print(df[['id_jugador', 'jugador_mean_dist', 'jugador_mean_load', 'jugador_mean_acc', 'jugador_mean_dec']].head())
df.columns.tolist()

✅ Nuevas columnas añadidas: ['Pos_Defensor', 'Pos_Delantero', 'Pos_Mediocampista', 'Lin_Defensa Central', 'Lin_Defensa Lateral', 'Lin_Delantera', 'Lin_Extremo', 'Lin_Medio Defensivo', 'Lin_Medio Ofensivo']
   id_jugador  jugador_mean_dist  jugador_mean_load  jugador_mean_acc  \
0          25        3854.093023          52.697674         28.325581   
1          25        3854.093023          52.697674         28.325581   
2          25        3854.093023          52.697674         28.325581   
3          25        3854.093023          52.697674         28.325581   
4          25        3854.093023          52.697674         28.325581   

   jugador_mean_dec  
0          27.44186  
1          27.44186  
2          27.44186  
3          27.44186  
4          27.44186  


['id_jugador',
 'Fecha',
 'microciclo_next',
 'tipo_dia_next',
 'Tipo_Dia',
 'Edad',
 'Peso_kg',
 'Estatura_cm',
 'CT_total_actual',
 'CE_total_actual',
 'CS_total_actual',
 'CR_total_actual',
 'riesgo_suavizado_3d_actual',
 'entrenos_total_next',
 'descansos_total_next',
 'partidos_total_next',
 'descansos_pre_partido_next',
 'entrenos_pre_partido_next',
 'entrenos_post_partido_next',
 'tipo_semana_next',
 'Carga_Explosiva',
 'Carga_Sostenida',
 'Carga_Regenerativa',
 'Distancia_total',
 'HMLD_m',
 'HSR_abs_m',
 'Sprints_cantidad',
 'Acc_3',
 'Dec_3',
 'Player_Load',
 'RPE',
 'Sprints_vel_max_kmh',
 'jugador_mean_dist',
 'jugador_mean_load',
 'jugador_mean_acc',
 'jugador_mean_dec',
 'Pos_Defensor',
 'Pos_Delantero',
 'Pos_Mediocampista',
 'Lin_Defensa Central',
 'Lin_Defensa Lateral',
 'Lin_Delantera',
 'Lin_Extremo',
 'Lin_Medio Defensivo',
 'Lin_Medio Ofensivo']

In [9]:
# ============================================
# ⚙️ SELECCIÓN DE FEATURES Y TARGET
# ============================================

features = [
    # Contexto planificado (semana y día)
    'tipo_semana_next', 'tipo_dia_next',

    # Carga y estado actual
    'CT_total_actual', 'CE_total_actual', 'CS_total_actual', 'CR_total_actual',
    'riesgo_suavizado_3d_actual',

    # Estructura de la próxima semana
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next',

    # Contexto fisiológico del jugador
    'jugador_mean_dist', 'jugador_mean_load', 'jugador_mean_acc', 'jugador_mean_dec'
] + [c for c in df.columns if c.startswith('Pos_') or c.startswith('Lin_')]

target = 'Carga_Sostenida'

print("📊 Total de features:", len(features))


📊 Total de features: 25


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr

# Codificar tipo de semana y tipo de día
le_semana = LabelEncoder()
le_dia = LabelEncoder()

df['tipo_semana_next'] = le_semana.fit_transform(df['tipo_semana_next'])
df['tipo_dia_next'] = le_dia.fit_transform(df['tipo_dia_next'].astype(str))

# Separar features y target
X = df[features]
y = df[target]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modelo
rf = RandomForestRegressor(n_estimators=300, max_depth=12, random_state=42)
rf.fit(X_train, y_train)

# Predicciones
y_pred = rf.predict(X_test)

# Evaluaciones
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
spearman_corr, _ = spearmanr(y_test, y_pred)

print("📈 R²:", round(r2, 3))
print("📊 MAE:", round(mae, 2))
print("🔗 Correlación de rangos (Spearman):", round(spearman_corr, 3))


📈 R²: 0.45
📊 MAE: 194.66
🔗 Correlación de rangos (Spearman): 0.689


In [11]:
import joblib, os

output_dir = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/src/chivas_ml/ml/registry/modelo_clas_CS"
os.makedirs(output_dir, exist_ok=True)

# Guardar modelo y objetos asociados
joblib.dump(rf, os.path.join(output_dir, "model_rf_CS_tendencias.pkl"))
joblib.dump(scaler, os.path.join(output_dir, "scaler_CS.pkl"))

print("✅ Modelo, scaler y label encoders guardados correctamente.")


✅ Modelo, scaler y label encoders guardados correctamente.



# 💪 Modelo de Predicción de Carga — CS (Carga Sostenida)

## 🎯 Objetivo
Desarrollar un modelo que **prediga la carga sostenida (CS)** de los entrenamientos diarios, basada en las condiciones semanales, la estructura del microciclo y las características fisiológicas de cada jugador.

La **carga sostenida** refleja el trabajo prolongado de intensidad media-alta, asociado al componente aeróbico y la resistencia física.  
Este modelo busca **detectar tendencias de mantenimiento o acumulación de carga**, fundamentales para ajustar las cargas totales semanales y prevenir la fatiga.

---

## ⚙️ Datos utilizados
- **Origen:** Vista extendida del modelo de carga diaria.
- **Filtrado:** Solo días de entrenamiento (sin partidos ni descansos).
- **Variables principales:**
  - `Tipo_Semana` (baja / media / alta)
  - `Tipo_Dia` (+1, +2, -1, -2, +3, etc.)
  - `CT_total`, `CE_total`, `CR_total` — cargas semanales acumuladas
  - `Riesgo_Suavizado_3d` — indicador de fatiga y riesgo
  - `Edad`, `Peso_kg`, `Estatura_cm`, `Posicion_…`
  - `jugador_mean_*` — medias históricas individuales de rendimiento
- **Variable objetivo (target):** `CS`

---

## 🧩 Metodología
- Modelo: `RandomForestRegressor`
- División: `train_test_split (80/20)`
- Escalado: `StandardScaler`
- Métricas de evaluación:
  - Coeficiente de determinación (**R²**)
  - Error absoluto medio (**MAE**)
  - Correlación de rangos (**Spearman ρ**)

---

## 📊 Resultados obtenidos

| Métrica | Valor |
|----------|--------|
| **R²** | 0.45 |
| **MAE** | 194.66 |
| **Correlación Spearman (ρ)** | 0.689 |

---

## 🔍 Interpretación

🔹 **R² (0.45):** El modelo explica el 45 % de la variación total en la carga sostenida, lo cual representa una **capacidad predictiva alta** para una métrica dependiente del volumen total de entrenamiento.  
🔹 **MAE (194.66):** Error medio razonable, coherente con la escala típica de valores de carga sostenida (500–1500).  
🔹 **ρ = 0.689:** Excelente correlación de rangos; el modelo **entiende la tendencia fisiológica sostenida**, distinguiendo correctamente los días y semanas de mayor o menor carga volumétrica.

---

## 📈 Conclusiones
- El modelo **captura fielmente las dinámicas de carga sostenida** asociadas a microciclos de mantenimiento o acumulación.  
- Puede utilizarse para **predecir cargas aeróbicas semanales** y planificar microciclos equilibrados según el tipo de semana.  
- Complementa al modelo de **Carga Explosiva (CE)**, ofreciendo una visión dual de la carga física global.

---

## 💾 Artefactos del modelo
Ubicación:  
`/src/chivas_ml/ml/registry/modelo_cs/`

Archivos generados:

model_rf_cs_tendencias.pkl

scaler_cs.pkl


---

## 🧠 Próximos pasos
- Integrar este modelo junto con `CE` y `CR` para obtener una **clasificación dinámica de la carga dominante** (Explosiva, Sostenida, Regenerativa).  
- Evaluar versiones del modelo con datos acumulados de los últimos 3–5 microciclos (tendencia longitudinal).  
- Probar arquitectura **Gradient Boosting Regressor** para mejorar precisión sin perder interpretabilidad.

---

## 💡 Valor práctico
Este modelo permite al preparador físico:
- **Ajustar volúmenes semanales** para evitar acumulación excesiva de carga sostenida.  
- **Planificar microciclos equilibrados**, alternando entre fases de mantenimiento, ajuste y regeneración.  
- **Predecir la carga total esperada** según la planificación de entrenamientos y tipo de semana.
